In [12]:
import parselmouth
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import pandas as pd
import subprocess
import time

from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.neural_network import MLPClassifier

In [5]:
# the differnt emotions in the files
emotions = ['anxiety', 'boredom', 'cold-anger', 'contempt', 'despair', 'disgust', 'elation', 'happy',
            'hot-anger', 'interest', 'neutral', 'panic',    'pride',    'sadness', 'shame']
clr =      ["red",  "cyan",   "orange",  "purple",   "magenta", "green",    "yellow",  "pink",
            "yellow", "green", "magenta", "purple",  "orange", "cyan",      "red"]

speakers = ['mf_001', 'cc_001', 'jg_001', 'mm_001', 'cl_001', 'gg_001', 'mk_001']

In [6]:
# directory to opensmile
smile = "/opensmile-2.3.0/inst/bin"
cwd = os.getcwd()
toFiles = cwd + "/speech_files/"


run = './SMILExtract'
conf = '../../config/IS09_emotion.conf'
output = "myfeatureset.csv"

In [ ]:
# saving the names of the files
names = []
# change directory to opensmile
os.chdir(smile)
# batching processing the audio file
# all the files should be contained in the folder called speech_files
for wave_file in glob.glob(toFiles + "*.wav"):
    
    name = wave_file.split("/")[-1][:len(wave_file.split("/")[-1])-4]
    print("Processing {}...".format(name))
#     print(run , '-C' , conf ,  '-I' , wave_file, '-csvoutput', output)
    p = subprocess.Popen([run , '-C' , conf ,  '-I' , wave_file, '-csvoutput', output])
    names.append(name)
    p.wait()

# reading the final feature file    
data = pd.read_csv(output, sep=';')

#changing names of the file
print("number of files processed and featuresized: ", len(names))
for x in range(len(names)):
    data['name'][x] = names[x]

# change back directory
os.chdir(cwd)

#saving file
data.to_csv(output, index=False)

In [7]:
data = pd.read_csv(output)

In [9]:
# speaker level data dictionary
sdata = {}
for spk in speakers:
    sdata[spk] = [[],[]]
    
count = 0
for index, row in data.iterrows():

    name = row['name'].split('_')
    sp = name[0] + "_" + name[1]
    emot = name[2]
#     print(sp, emot)
    
    sdata[sp][0].append(emot)
    sdata[sp][1].append(list(row[1:]))
    count+=1

print("datapoints: ", count )

datapoints:  2324


In [10]:
reports = {'p':[], 'nb':[], 'r':[], 'c':[], 's':[]}

In [13]:
# one speaker out cross validation
# get the data for test and train
for spk in speakers:
    
    test = [[],[]]
    train = [[],[]]
    
    for sp in speakers:
        if sp == spk:
            test[0] = test[0] + sdata[sp][0]
            test[1] = test[1] + sdata[sp][1]
        else:
            train[0] = train[0] + sdata[sp][0]
            train[1] = train[1] + sdata[sp][1]

    print('test speaker: ', spk)

    testX = np.array(test[1])
    trainX = np.array(train[1])
    

    trainY = []
    testY = []

    for x in range(len(train[0])):
        e = train[0][x]
        trainY.append(emotions.index(e))

    for x in range(len(test[0])):
        e = test[0][x]
        testY.append(emotions.index(e))

    trainY = np.array(trainY)
    testY = np.array(testY)


    

    #multilayer perceptron
    mlf = MLPClassifier(hidden_layer_sizes=(250,500,150, 100), activation='tanh',
                        max_iter=500, alpha=0.0001,learning_rate="adaptive",
                        learning_rate_init=0.0001,solver='sgd', verbose=False,  
                        random_state=110,tol=0.000000000001, nesterovs_momentum=True, warm_start=True)
   
    mlf.fit(trainX, trainY)
    y_pred = mlf.predict(testX)
    print('perceptron', spk)
    rep = classification_report(testY, y_pred, target_names=emotions)
    print(rep)
    reports['p'].append(rep)
    
    
#     # SVM
#     clf = LinearSVC(random_state=0, tol=1e-5, max_iter=2000)
#     clf.fit(trainX, trainY)
#     y_pred = clf.predict(testX)
#     print(classification_report(testY, y_pred, target_names=emotions))
  
    
    #Ridge Regression
    rlf = RidgeClassifier( max_iter=100000).fit(trainX, trainY)
    y_pred = rlf.predict(testX)
    print('Ridge', spk)
    rep = classification_report(testY, y_pred, target_names=emotions)
    print(rep)
    reports['r'].append(rep)
    
    # Naive Bayes Gaussian
    glf = GaussianNB().fit(trainX, trainY)
    y_pred = glf.predict(testX)
    print('naive bayes', spk)
    rep = classification_report(testY, y_pred, target_names=emotions)
    print(rep)
    reports['nb'].append(rep)
    
    
    # nearest centroid
    cnlf = NearestCentroid().fit(trainX, trainY)
    y_pred = cnlf.predict(testX)
    print('centroid', spk)
    rep = classification_report(testY, y_pred, target_names=emotions)
    print(rep)
    reports['c'].append(rep)
    

test speaker:  mf_001
perceptron mf_001
              precision    recall  f1-score   support

     anxiety       0.17      0.23      0.19        22
     boredom       0.22      0.22      0.22        27
  cold-anger       0.17      0.05      0.08        20
    contempt       0.11      0.02      0.04        44
     despair       0.00      0.00      0.00        16
     disgust       0.00      0.00      0.00         1
     elation       0.00      0.00      0.00        26
       happy       0.13      0.87      0.22        23
   hot-anger       0.00      0.00      0.00        21
    interest       0.08      0.05      0.06        19
     neutral       0.00      0.00      0.00        10
       panic       0.00      0.00      0.00        12
       pride       0.00      0.00      0.00        18
     sadness       0.25      0.15      0.19        20
       shame       0.10      0.10      0.10        20

   micro avg       0.13      0.13      0.13       299
   macro avg       0.08      0.11      0

/home/abdullah/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abdullah/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


perceptron cc_001
              precision    recall  f1-score   support

     anxiety       0.05      0.30      0.09        10
     boredom       0.00      0.00      0.00        15
  cold-anger       0.00      0.00      0.00        15
    contempt       0.00      0.00      0.00        22
     despair       0.00      0.00      0.00         9
     disgust       0.00      0.00      0.00        31
     elation       0.00      0.00      0.00        16
       happy       0.11      0.52      0.18        23
   hot-anger       0.00      0.00      0.00        14
    interest       0.06      0.06      0.06        17
     neutral       0.00      0.00      0.00        18
       panic       0.00      0.00      0.00        18
       pride       0.00      0.00      0.00        23
     sadness       0.05      0.08      0.06        13
       shame       0.08      0.10      0.09        21

   micro avg       0.07      0.07      0.07       265
   macro avg       0.02      0.07      0.03       265
weighted

/home/abdullah/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abdullah/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


perceptron jg_001
              precision    recall  f1-score   support

     anxiety       0.11      0.05      0.07        19
     boredom       0.00      0.00      0.00        14
  cold-anger       0.00      0.00      0.00        22
    contempt       0.00      0.00      0.00        23
     despair       0.50      0.05      0.09        21
     disgust       0.00      0.00      0.00        23
     elation       0.00      0.00      0.00        20
       happy       0.08      0.90      0.15        20
   hot-anger       0.00      0.00      0.00        18
    interest       0.03      0.05      0.04        19
     neutral       0.00      0.00      0.00         8
       panic       0.00      0.00      0.00        14
       pride       0.00      0.00      0.00        18
     sadness       0.00      0.00      0.00        19
       shame       0.33      0.07      0.11        15

   micro avg       0.08      0.08      0.08       273
   macro avg       0.07      0.07      0.03       273
weighted

...

/home/abdullah/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/abdullah/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


perceptron cl_001
              precision    recall  f1-score   support

     anxiety       0.11      0.67      0.19        21
     boredom       0.19      0.17      0.18        29
  cold-anger       0.00      0.00      0.00        27
    contempt       0.07      0.04      0.05        25
     despair       0.00      0.00      0.00        29
     disgust       0.00      0.00      0.00        22
     elation       0.00      0.00      0.00        27
       happy       0.14      0.62      0.22        21
   hot-anger       0.00      0.00      0.00        26
    interest       0.05      0.04      0.04        26
     neutral       0.00      0.00      0.00        17
       panic       0.00      0.00      0.00        21
       pride       0.00      0.00      0.00        24
     sadness       0.03      0.04      0.04        27
       shame       0.17      0.31      0.22        26

   micro avg       0.12      0.12      0.12       368
   macro avg       0.05      0.13      0.06       368
weighted

/home/abdullah/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abdullah/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


perceptron mk_001
              precision    recall  f1-score   support

     anxiety       0.22      0.07      0.11        29
     boredom       0.00      0.00      0.00        20
  cold-anger       0.00      0.00      0.00        23
    contempt       0.12      0.05      0.07        21
     despair       0.00      0.00      0.00        53
     disgust       0.00      0.00      0.00        21
     elation       0.06      1.00      0.12        23
       happy       0.00      0.00      0.00        42
   hot-anger       0.00      0.00      0.00        22
    interest       0.00      0.00      0.00        44
     neutral       0.00      0.00      0.00         8
       panic       0.00      0.00      0.00        21
       pride       0.00      0.00      0.00        23
     sadness       0.00      0.00      0.00        22
       shame       0.00      0.00      0.00        25

   micro avg       0.07      0.07      0.07       397
   macro avg       0.03      0.07      0.02       397
weighted

/home/abdullah/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/abdullah/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abdullah/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abdullah/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', '